In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy
import scipy
import matplotlib
import matplotlib.dates as mpd
import pylab as plt
import datetime as dtm
import pytz
import multiprocessing as mpp
import pickle
import os
import time
import numba
#
import pyspark
#
# TODO: phase out unreferenced hpc_lib calls...
import hpc_lib

#
#data_file_name = 'data/mazama_usage_20200506_tool8.out'
#data_file_name = 'data/sacct_sherlock_out_serc2020_05_08.out'
data_file_name = 'data/serc_usage_20200914.out'
#
pkl_name = "{}.pkl".format(os.path.splitext(data_file_name)[0])
h5_name = "{}.h5".format(os.path.splitext(data_file_name)[0])
#

In [ ]:
conf = pyspark.SparkConf('local[*]').set("spark.cores.max", "6").set("spark.executor.instances", "4").set("spark.executor.cores","2")
conf = conf.set("spark.executor.memory", "4g").set("spark.executor.pyspark.memory", "3g")
sc   = pyspark.SparkContext(conf=conf)
#

In [11]:
# also build a SQL context?
sc_sql = pyspark.SQLContext(sc)
spark = pyspark.sql.SparkSession.builder.appName('HPC_loader').getOrCreate()

In [4]:
types_dict = hpc_lib.SACCT_data_handler.default_types_dict

@numba.jit
def process_row(rw, delim='|'):
    # use this with MPP processing:
    # ... but TODO: it looks like this is 1) inefficient and 2) breaks with large data inputs because I think it pickles the entire
    #  class object... so we need to move the MPP object out of class.
    #
    # use this for MPP processing:
    rws = rw.split(delim)
    #return [None if vl=='' else self.types_dict.get(col,str)(vl)
    #            for k,(col,vl) in enumerate(zip(self.headers, rw.split(self.delim)[:-1]))]
    return [None if vl=='' else types_dict.get(col,str)(vl)
                for k,(col,vl) in enumerate(zip(self.headers, rws[:-1]))] + [rws[self.RH['JobID']].split('.')[0]]


In [12]:
# using the SparkSession, we can load the csv directly to a spark Data Frame:
df_rows = spark.read.csv(data_file_name, header=True, sep='|')
#
print('** ', df_rows.dtypes)
print('** ', df_rows.head)
#
print('*** *** ')
for rw in df_rows.take(5):
    print('** ', rw)

In [5]:
def f_rw(rw):
    rws = rw.split('|')
    #
    if not len(rws)==0:
        return rws
    
    

In [6]:
lines = sc.textFile(data_file_name)

delim = '|'

rows = lines.map(f_rw)

In [7]:
rows.persist(StorageLevel.MEMORY_AND_DISK)
all_rows = rows.collect()
rows.unpersist()

#grouped = rows.filter(lambda rw:rw[0]!='User').groupBy(lambda rw: rw[0])

NameError: name 'StorageLevel' is not defined

In [ ]:
n_rws = rows.count()
print('** n_rws: {}'.format( n_rws) )
#
n=1000
xx = rows.take(n)

In [ ]:
#print(type(grouped))
xx = grouped.take(100)

In [ ]:
# print('** lines: ')
# for ln in lines.take(10):
# 	print('** **: ', ln)
#
print('rows: ')
for rw in rows.take(10):
	print('** **: ', rw)
#
#
# print('** rows again: ')
# for rw in rows.take(10):
#         print('** **: ', rw)
#
print('** groups: ')
for rw in grouped.take(10):
    print('* *: ', rw)

In [ ]:
print('begin all_rows[]: ')
t0 = time.time()
#
all_rows = rows.collect()
#
print('** time: {}'.format(time.time()-t0))

In [ ]:
print('begin all_lines: ')
t0 = time.time()
#
all_lines = lines.collect()
#
print('** time: {}'.format(time.time()-t0))

In [ ]:
print('** sc: ', sc.defaultParallelism)